<a href="https://colab.research.google.com/github/dave20012/pixray-demo-with-local-save-enabled./blob/main/Original_of_public_pixray_demo_with_local_save_enabled_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pixray demo with local save enabled


The ability to **save models/checkpoints to google drive** has been added! This means you dont have to re-download them each time you start the notebook.

It also now saves image results to 'MyDrive/pixray_generated_art/*(prompts)*.png'

In [ ]:
#@title Setup

#@markdown Please execute this cell by pressing the _Play_ button 
#@markdown on the left. You should only need to run this part once.

google_drive = True #@param {type:"boolean"}

#@markdown You can use your mounted Google Drive to cache the model files.  This will save time (and resources!) when you re-visit this notebook in the future.

#@markdown Click here if you'd like to save the model checkpoint files to your Google Drive:
yes_please = True #@param {type:"boolean"}

#@markdown When complete you will need to do Runtime -> Restart Runtime from the menu

#
from google.colab import drive

# Add a gpu check
nvidia_output = !nvidia-smi --query-gpu=memory.total --format=noheader,nounits,csv
gpu_memory = int(nvidia_output[0])
if gpu_memory < 14000:
  print(f"--> GPU check: ONLY {gpu_memory} MiB available: WARNING, some things might not work <--")
else:
  print(f"GPU check: {gpu_memory} MiB available: this should be fine")

print("Installing...")
from IPython.utils import io
with io.capture_output() as captured:
  !rm -Rf pixray
  !git clone --recursive https://github.com/pixray/pixray
  !pip install -r pixray/requirements.txt
  !pip uninstall -y tensorflow 
  !git clone https://github.com/pixray/diffvg
  %cd diffvg
  !git submodule update --init --recursive
  !python setup.py install
  %cd ..
  !pip freeze | grep torch


if google_drive:
    from google.colab import drive
    drive.mount('/content/drive')


if yes_please:
    #models dir is hardcoded to models/. change to google drive storage:
    !sed -i 's/models\//\/content\/drive\/MyDrive\/models\//g' pixray/vqgan.py 
    !sed -i 's/\"models\"/\"\/content\/drive\/MyDrive\/models\"/g' vqgan.py
    !mkdir /content/drive/MyDrive/models
    !mkdir /content/drive/MyDrive/pixray_generated_art
    model_path = '/content/drive/MyDrive/models'
    print("Downloading wikiart model...")
    !wget --continue 'https://github.com/pixray/pixray/releases/download/v1.7.1/vqgan_wikiart_16384.ckpt' -O "/content/drive/MyDrive/models/vqgan_wikiart_16384.ckpt"

import os
if not os.path.isfile("first_init_complete"):
  # put stuff in here that should only happen once
  os.mknod("first_init_complete")
else:
  print("Please choose Runtime -> Restart Runtime from the menu to continue! IT WILL FAIL with 'PIL.TiffTags' has no attribute 'IFD' if you dont.")
  print("Setup Complete! Good luck with your drawing")


In [ ]:
#@title Easy Settings

#@markdown Let's take a prompt and try drawing with a number
#@markdown of different models. The models we will be using are
#@markdown VQGAN models which mainly differ on the trained data
#@markdown and architecture. Swapping the model will make the result
#@markdown look differnt. For example: "wikiart" is trained on
#@markdown artwork and so draws things less photorealistic.

#@markdown Best bet: just try different models out and see what happens.

import sys
sys.path.append("pixray")

prompts = "flower mandala:2|detailed painting:1|by charlotte harding:1|landscape poster HQ artstation:.6|blur:-.5|abstract:-.5" #@param {type:"string"}

vqgan_model = "wikiart_16384" #@param ["imagenet_f16_16384", "imagenet_f16_1024", "openimages_f16_8192", "wikiart_1024", "wikiart_16384", "coco"]

quality = "draft" #@param ["draft", "normal", "better", "best"]

iterations = 120 #@param [120,200,300]

aspect = "portrait" #@param ['portrait', 'widescreen', 'square']

#@markdown When you have the settings you want, press the play button on the left.
#@markdown We will apply these settings below.

## Apply settings

The code block below shows how the settings above get applied. You can add your own before we start the code. For example, you can set the drawer to "pixel" or used a fixed random number seed. This allows us to mix and match the best settings discussed in differnt notebooks.

Make sure you click the play button below *after* running the easy settings above if you want to make any of these modifications before running.

In [ ]:
cd /content/drive/MyDrive/models


In [ ]:
    !wget --continue 'https://github.com/pixray/pixray/releases/download/v1.7.1/vqgan_wikiart_16384.ckpt' -O "/content/drive/MyDrive/models/vqgan_wikiart_16384.ckpt"


In [ ]:
keywords = keywords = ['20 megapixels', '2d game art', '32k huhd', '3d render', '4k', '8k 3d', '8k resolution', 'a b frost', 'aaron douglas', 'aaron miller', 'abstract', 'academic art', 'acrylic art', 'adam paquette', 'adam rex', 'aftereffects', 'agnes lawrence pelton', 'ahmed karahisari', 'airbrush art', 'akihiko yoshida', 'alan pollack', 'albert bierstadt', 'albert dorne', 'albert joseph p not', 'albert namatjira', 'alberto giacometti', 'alberto vargas', 'albrecht d rer', 'aldus manutius', 'aleksander gierymski', 'aleksi briclot', 'alesso baldovinetti', 'alex grey', 'alex horley orlandelli', 'alex katz', 'alex ross', 'alex toth', 'alexander sharpe ross', 'alfred charles parker', 'alfred freddy krupa', 'allen tupper true', 'alphonse mucha', 'ambient occlusion', 'american propaganda', 'amos sewell', 'amy weber', 'anarogynous', 'anato finnstark', 'andr fran ois', 'andr masson', 'andrew loomis', 'andrew robinson', 'andrew wyeth', 'andy warhol', 'anime', 'anish kapoor', 'anita kunz', 'anna and elena balbusso', 'anna haifisch', 'anna mary robertson moses', 'anni albers', 'ansel adams', 'anson maddocks', 'anthony palumbo', 'anthony s waters', 'anton otto fischer', 'antonio mancini', 'aquirax uno', 'archibald motley jr', 'arcimboldo', 'arnie swekel', 'art deco', 'art nouveau', 'art on instagram', 'art spiegelman', 'artemisia gentileschi', 'arthur burdett frost', 'arthur garfield dove', 'arthur rackham', 'artstation hd', 'artstation hq', 'artwork', 'associated press photo', 'aubrey beardsley', 'aur l bern th', 'austin briggs', 'austin english', 'avant-garde', 'ayako rokkaku', 'ayami kojima', 'ayshia ta k n', 'banksy', 'barbara nessim', 'baroque', 'barron storey', 'bascove', 'bastien l deharme', 'bauhaus', 'bayard wu', 'beatrix potter', 'beeple render', 'behance hd', 'beksinski', 'ben stahl', 'ben thompson', 'ben zoeller', 'benjamin franklin', 'benjamin marra', 'bernard d', 'bernie d andrea', 'bikash bhattacharjee', 'bill sienkiewicz', 'bill ward', 'bill watterson', 'bob eggleton', 'bob ross', 'bob singer', 'bokeh', 'booru', 'boris vallejo', 'bourgeois', 'brad holland', 'brenda chamberlain', 'brian bolland', 'brian chippy dugan', 'brian fies', 'brian froud', 'brian sn ddy', 'bridget riley', 'brigette barrager', 'brom', 'bruce timm', 'bryce 3d', 'bunny yeager', 'by edward hopper', 'by james gurney', 'by jim burns', 'by thomas kinkade', 'by vincent di fate', 'c zanne', 'cam sykes', 'camille corot', 'camille pissarro', 'canaletto', 'caravaggio', 'carl barks', 'carl critchlow', 'carla wyzgala', 'carles delclaux is', 'carlos saenz de tejada', 'caroline mytinger', 'cedric peyravernay', 'cgsociety', 'chagall', 'chalk art', 'charcoal drawing', 'charles addams', 'charles dana gibson', 'charles marion russell', 'charles martin', 'charles schulz', 'charlotte harding', 'chase stone', 'chiaroscuro', 'child s drawing', 'chinese style art', 'chippy', 'chris cold', 'chris foss', 'chris moore', 'chris rahn', 'chris rallis', 'chris spollen', 'chris ware', 'christian august lorentzen', 'christopher moeller', 'christopher rush', 'christopher williams', 'christopher wren', 'cindy sherman', 'claire hummel', 'clara miller burd', 'clara peeters', 'claude monet', 'cliff childs', 'clint cearley', 'cluttered', 'coby whitmore', 'coles phillips', 'color field painting', 'colorized', 'commision for', 'concept art', 'constructivism', 'contemporary', 'contrasting light', 'cornelis bisschop', 'craig mullins', 'craig thompson', 'creative commons attribution', 'creepypasta', 'cricorps gr goire', 'criterion collection', 'cryengine', 'cubism', 'cynthia sheppard', 'd alexander gregory', 'd rer', 'da vinci', 'daarken', 'dadaism', 'dai jin', 'dal', 'dali', 'damien hirst', 'dan content', 'dan frazier', 'dan luvisi', 'dan scott', 'dan smith', 'daniel gelon', 'daniel ljunggren', 'daniel merriam', 'darek zabrocki', 'daren bader', 'darrell riche', 'datamosh', 'dave allsop', 'dave arredondo', 'dave dorman', 'dave gibbons', 'dave kendall', 'dave mckean', 'dave melvin', 'david begbie', 'david firth', 'david macaulay', 'david martin', 'david palumbo', 'david small', 'dc comics', 'dean cornwell', 'deep black accents', 'delaunay', 'depth of field', 'derf', 'desaturated', 'destructive', 'detailed painting', 'detailed', 'deviantart hd', 'deviantart', 'diane dillon', 'dick bickenbac', 'dick bickenbach', 'dicky doyle', 'diego gisbert llorens', 'diego velazquez', 'digital illustration', 'digitally enhanced', 'domirinic fegallia', 'don arday', 'donato giancola', 'dong kingman', 'dorothea tanning', 'dorothy hood', 'douard detaille', 'douard manet', 'doug wildey', 'douglas shuler', 'dr seuss', 'drew tucker', 'dslr', 'dutch golden age', 'dye transfer', 'e h shepard', 'e simms campbell', 'earle bergey', 'earnst haeckel', 'ed benedict', 'ed roth', 'edgar degas', 'edmund dulac', 'edmund f ward', 'edvard munch', 'edward gorey', 'edward hopper', 'edward kemble', 'edward lear', 'edward p beard jr', 'edward sorel', 'edwin austin abbey', 'edwin deakin', 'edwin georgi', 'egon schiele', 'egyptian art', 'el greco', 'elaine duillo', 'eleanor vere boyle', 'elizabeth murray', 'elizabeth shippen green', 'elwood h smith', 'emanuel schongut', 'emma rios', 'epic', 'epsylon point', 'eric auld', 'eric dinyer', 'eric peterson', 'ernst fuchs', 'ernst', 'etienne delessert', 'eug ne delacroix', 'eugene leroy', 'eugeniusz zak', 'evaline ness', 'everett raymond kinstler', 'everett shinn', 'exekias', 'expressionism', 'eyvind earle', 'f scott hess', 'fairfield porter', 'fanart', 'fantasy art', 'fauvism', 'featured on artstation', 'fede galizia', 'felix octavius carr darley', 'feminine', 'feng zhu', 'film quality', 'film', 'filmic', 'fine art', 'fiona stephenson', 'fitz henry lane', 'flat shading', 'flemish baroque', 'flickering light', 'flickr', 'frances jetter', 'francesco furini', 'francis bacon', 'francis souza', 'francisco goya', 'franciszek starowieyski', 'frank frazetta', 'frank miller', 'frank schoonover', 'frank xavier leyendecker', 'franklin booth', 'franklin carmichael', 'franz vohwinkel', 'fred ludekens', 'fred marcellino', 'fred williams', 'frederick mccubbin', 'frida kahlo', 'fugot render', 'furaffinity', 'futurism', 'futuristic style', 'fuyuko matsui', 'gabor szikszai', 'galen dara', 'gary panter', 'gaudi', 'gaugin', 'genderless', 'geometric', 'george abe', 'george b bridgman', 'george cruikshank', 'george inness', 'george luks', 'georges seurat', 'georgia o keeffe', 'giacomo balla', 'giger', 'gil elvgren', 'gilles beloeil', 'gillis rombouts', 'giorgio de chirico', 'giorgione', 'giovanni battista gaulli', 'giuseppe camuncoli', 'glen angus', 'glitchy', 'global illumination', 'gloria stoll karn', 'glossy', 'glowing neon', 'gordon browne', 'grandma moses', 'graphic design', 'great composition', 'greg hildebrandt', 'greg rutkowski', 'greg spalenka', 'greg staples', 'gregorio lazzarini', 'gregory manchess', 'groovy', 'grzegorz rutkowski', 'gu hongzhong', 'guillermo del toro', 'guo xi', 'gustaf tenggren', 'gustav dor', 'gustav klimt', 'gustave dore', 'gwen john', 'h r giger', 'h r millar', 'haddon sundblom', 'hall of mirrors', 'hanna barbera', 'hannah h ch', 'hans hinterreiter', 'harlem renaissance', 'harold von schmidt', 'harriet powers', 'harrison fisher', 'harry beckhoff', 'harry haenigsen', 'harvey dunn', 'harvey kurtzman', 'hd mod', 'hdr', 'heather hudson', 'heinz edelman', 'henri matisse', 'henri rousseau', 'henry moore', 'henry raleigh', 'hexhue colors', 'hideyuki kikuchi', 'hieronim bosch', 'high quality photo', 'hilma af klint', 'hiromu arakawa', 'hisui sugiura', 'hokusai', 'holographic', 'howard chandler christy', 'howard knotts', 'howard lyon', 'howard pyle', 'hyper realism', 'hyperrealism', 'ian miller', 'ignacy witkiewicz', 'igor kieryluk', 'igor morski', 'ilford hp5', 'illustration', 'ilya kuvshinov', 'ilya repin', 'impressionism', 'indian style art', 'infrared', 'ink drawing', 'instax', 'irene and laurette patten', 'iridescent', 'isaac levitan', 'iso 200', 'israel tsvaygenbaum', 'istvan banyai', 'it jakuch', 'ivan aivazovsky', 'ivan bilibin', 'ivan shishkin', 'iwao takamoto', 'izzy medrano', 'j c leyendecker', 'j frederick smith', 'j zef mehoffer', 'jacek yerka', 'jack kirby', 'jackson pollock', 'jacques h rold', 'jacqui morgan', 'jaime jones', 'james gillray', 'james gurney', 'james jean', 'james jonathan jeffries', 'james montgomery flagg', 'james paick', 'james ryman', 'james warhola', 'james wood', 'jamie hewlett', 'jan brett', 'jan kip', 'jan miense molenaer', 'jan van eyck', 'janet fish', 'japanese style art', 'jason a engle', 'jason chan', 'jason felix', 'jasper johns', 'jay hambidge', 'jean cunningham', 'jean giraud', 'jean metzinger', 'jean michel basquiat', 'jeff a menges', 'jeff koons', 'jeff miracola', 'jeffrey smith', 'jennifer janesko', 'jerry eisenberg', 'jerry pinkney', 'jesper ejsing', 'jesper myrfors', 'jessie willcox smith', 'jim burns', 'jim lee', 'jim murray', 'jim nelson', 'jim steranko', 'jimmy ernst', 'joan mir', 'joan snyder', 'joan tuset', 'joaqu n sorolla', 'jodorowsky', 'joe bowler', 'joe de mers', 'joe shuster', 'joe sorren', 'joel shapiro', 'johan grenier', 'johann bodin', 'johann kretzschmer', 'johannes helgeson', 'johannes vermeer', 'johannes voss', 'john atherton', 'john atkinson grimshaw', 'john avon', 'john backderf', 'john berkey', 'john broadley', 'john carpenter', 'john frederick kensett', 'john french sloan', 'john george sowerby', 'john harris', 'john henry kingsley', 'john j park', 'john la gatta', 'john martin', 'john matson', 'john philip falter', 'john romita jr', 'john wilson', 'jon foster', 'jon whitcomb', 'jonathan solter', 'jordan grimmer', 'jorge jacinto', 'joseph bowler', 'joseph clement coll', 'joseph cornell', 'joseph keppler', 'josh bayer', 'joyce ballantyne brand', 'juan gim nez', 'juan gris', 'julian allen', 'jung park', 'junji ito', 'justin sweet', 'kadir nelson', 'kahlo', 'kaja foglio', 'kandinsky', 'kanzan shimomura', 'karl kopinski', 'karl thiart', 'kate greenaway', 'katsushika i', 'kawanabe ky sai', 'kay nielsen', 'kazimir malevich', 'keith haring', 'kelly mckernan', 'keren katz', 'kev walker', 'kinuko craft', 'klimt', 'kodak ektra', 'kodak gold 200', 'kodak portra', 'kogan gengei', 'konstantin vasilyev', 'konstantinas ciurlionis', 'kusama', 'kyle lambert', 'ladr nn', 'lale westvind', 'lars grant west', 'laura muntz lyall', 'laurel burch', 'laurie lipton', 'lee loughridge', 'lena alexander', 'leo and diane dillon', 'leo lionni', 'leonardo da vinci', 'leonora carrington', 'li chevalier', 'li di', 'li tang', 'li tiefu', 'lichtenstein', 'lin tinggui', 'lisa frank', 'lisa yuskavage', 'liza donnelly', 'loic zimmerman', 'lomo', 'lorraine fox', 'louis comfort tiffany', 'louis glackens', 'louis janmot', 'louise bourgeois', 'low key', 'low poly', 'lowbrow', 'luca zontini', 'lucas graciano', 'lucian freud', 'ludwig bemelmans', 'luis paret y alcazar', 'luis royo', 'luma rouge', 'lyle tuttle', 'lynda benglis', 'lynn pauley', 'lyubov popova', 'm bius', 'm c escher', 'ma gorzata serwatka', 'mabel rollins harris', 'mac conner', 'maciej kuciara', 'macro photography', 'made of all of the above', 'made of beads and yarn', 'made of cardboard', 'made of cheese', 'made of crystals', 'made of feathers', 'made of flowers', 'made of glass', 'made of insects', 'made of liquid metal', 'made of mist', 'made of paperclips', 'made of plastic', 'made of rubber', 'made of trash', 'made of vines', 'made of wire', 'made of wrought iron', 'magali villeneuve', 'maginel wright enright barney', 'makoto aida', 'makuzu k zan', 'malczewski', 'malevich', 'malvin gray johnson', 'manjit bawa', 'marble sculpture', 'marc bell', 'marc simonetti', 'marc tetro', 'marcel duchamp', 'margaret brundage', 'margaret keane', 'margaret macdonald mackintosh', 'margareta sterian', 'marianne north', 'marie gabrielle capet', 'mario cooper', 'mark english', 'mark poole', 'mark ryden', 'mark tedin', 'mark zug', 'marshall arisman', 'martin johnson heade', 'martin schongauer', 'maruyama kyo', 'marvel comics', 'mary blair', 'mary cassatt', 'mary hallock foote', 'mary jane begin', 'masamune shirow', 'masculine', 'masterpiece', 'mathias kollros', 'mati klarwein', 'matt cavotta', 'matt stewart', 'matte background', 'matte drawing', 'matte painting', 'matte photo', 'matte', 'matthew d wilson', 'matthias weischer', 'mattise', 'maurice esteve', 'max ernst', 'max gubler', 'maxfield parrish', 'maximalist', 'mc escher', 'mead schaeffer', 'melissa a benson', 'melissa benson', 'micha klein', 'michael sutfin', 'michael whelan', 'michelangelo buonarotti', 'michelangelo', 'mikalojus konstantinas iurlionis', 'mike bierek', 'mike mignola', 'milton caniff', 'milton glaser', 'minecraft', 'miniature world', 'minimalism', 'minimalist', 'miwa komatsu', 'mixed media', 'miyamoto', 'modernism', 'moebius', 'mondrian', 'monet', 'mort k nstler', 'movie poster', 'munch', 'murakami', 'murray tinkelman', 'mustafa rakim', 'myportfolio', 'n c wyeth', 'nancy stahl', 'narayan shridhar bendre', 'natasha tan', 'nathalie rattner', 'national geographic photo', 'neal adams', 'ned m seidler', 'neil blevins', 'neil boyle', 'nelson alexander ross', 'neoclassicism', 'neon art', 'neysa mcmein', 'nicolette macnamara', 'niko henrichon', 'nil gleyen', 'nils hamm', 'noah bradley', 'norman lewis', 'norman rockwell', 'norman saunders', 'o keeffe', 'odilon redon', 'ohara koson', 'oil on canvas', 'oil painting', 'olga rozanova', 'olha darchuk', 'olivia de berardinis', 'osamu tezuka', 'pablo carpio', 'pablo dominguez', 'pablo picasso', 'pablo rey', 'panorama', 'paolo parente', 'parallax', 'pat oliphant', 'paul bril', 'paul cezanne', 'paul davis', 'paul gauguin', 'paul klee', 'paulus decker', 'pawe kluza', 'pearl frush', 'pencil sketch', 'penny patricia poppycock', 'pete venters', 'peter de s ve', 'peter de seve', 'peter fiore', 'peter helck', 'peter madsen', 'peter max', 'peter mohrbacher', 'peter paul rubens', 'peter rockwell', 'peter wells', 'phallic', 'phil foglio', 'phillip peter price', 'photo taken with ektachrome', 'photo taken with fugifilm superia', 'photo taken with nikon d750', 'photo taken with provia', 'photocollage', 'photocopy', 'photoillustration', 'photorealistic', 'pia fries', 'picasso', 'pierre bonnard', 'pierre mion', 'piet mondrian', 'pieter bruegel', 'pieter claesz', 'piranesi', 'pixelized', 'pixiv', 'playstation 5 screenshot', 'pointilism', 'polished', 'pollock', 'polycount', 'pop art', 'poster art', 'pre raphaelite', 'prerendered graphics', 'provia', 'pruett carter', 'ps1 graphics', 'psychedelic', 'qu leilei', 'quantum wavetracing', 'quinton hoover', 'r o blechman', 'rainbow colors', 'ralph horsley', 'ralph mcquarrie', 'randolph caldecott', 'randy gallegos', 'randy post', 'randy vargas', 'raphael', 'raymond swanland', 'realism', 'rebecca guay', 'reginald richard redford', 'rei kamoi', 'reimagined by industrial light and magic', 'religious art', 'rembrandt', 'remedios varo', 'ren xun', 'renaissance painting', 'rendered in cinema4d', 'rendered in maya', 'rene magritte', 'renoir', 'reuben nakian', 'ric estrada', 'richard doyle', 'richard hess', 'richard kane ferguson', 'richard randolph rubenstein', 'richard rockwell', 'richter', 'rick griffin', 'riusuke fukahori', 'rob alexander', 'rob gonsalves', 'robbie trevino', 'robert beatty', 'robert childress', 'robert crumb', 'robert fawcett', 'robert m cunningham', 'robert maguire', 'robert mcginnis', 'robert peak', 'robert weaver', 'roberto da matta', 'robt l williams', 'rockwell kent', 'rococo', 'roger ballen', 'rolf armstrong', 'ron spears', 'ron spencer', 'rose o neill', 'ross tran', 'rough', 'roz chast', 'rtx on', 'rube goldberg', 'rudolf freund', 'rudolph belarski', 'russell patterson', 'ruth deckard', 'ruth sanderson', 'ryan barger', 'ryan pancoast', 'ryan stegman', 'ryan yee', 'salvador dal', 'salvador dali', 'sam spratt', 'sandro botticelli', 'sarah gough adamson', 'sarah morris', 'sarper baran', 'saul steinberg', 'saul tepper', 'scott gustafson', 'scott m fischer', 'scott samuel summers', 'screenshotsaturday', 'seapunk', 'seb mckinnon', 'shaddy safadi', 'sheikh hamdullah', 'sheilah beckett', 'shekhar gurera', 'shiny', 'shog janit', 'shot on 70mm', 'sidney richard percy', 'siegfried haas', 'simon bisley', 'sir john tenniel', 'slawomir maniak', 'smooth', 'sonia delaunay', 'soviet propaganda', 'speedpainting', 'spitpaint', 'ssao', 'stained glass', 'stan and jan berenstain', 'stan galli', 'stanis aw wyspia ski', 'stevan dohanos', 'steve argyle', 'steve brodner', 'steve dillon', 'steve ditko', 'steve prescott', 'steven belledin', 'steven james petruccio', 'stipple', 'stock photo', 'storybook illustration', 'studio ghibli', 'sunil das', 'sunrays shine upon it', 'suprematism', 'surrealism', 'susan heidi', 'susy pilgrim waters', 'svetlin velinov', 'syd mead', 'sydney prior hall', 'symbolism', 't s sullivant', 'takashi murakami', 'takato yamamoto', 'takeshi obata', 'tamara lempicka', 'taravat jalali farahani', 'taro okamoto', 'tatsuyuki tanaka', 'tattoo', 'ted degrazia', 'teresa fasolino', 'terese nielsen', 'terragen', 'terry redlin', 'thomas blackshear', 'thomas crane', 'thomas dalziel', 'thomas fogarty', 'thomas kinkade', 'thomas kluge', 'thomas m baxa', 'thomas nast', 'thomas rowlandson', 'thomas scholes', 'thornton oakley', 'thota vaikuntham', 'tilt shift', 'tim and greg hildebrandt', 'tim hildebrandt', 'titian', 'titus lunter', 'todd lockwood', 'tom la padula', 'tom lovell', 'tom w nerstrand', 'tomasz jedruszek', 'tomek setowski', 'tomer hanuka', 'tomi ungerer', 'tomokazu matsuyama', 'tony diterlizzi', 'tony sart', 'tony szczudlo', 'trending on artstation', 'trending on behance', 'trending on cgsociety', 'trending on deviantart', 'trending on dribbble', 'trending on pixiv', 'trending on wallup.net', 'tri x 400 tx', 'tuomas korpi', 'tyler edlin', 'tyler jacobson', 'ukiyo e', 'ultra hd', 'ultrafine detail', 'unreal engine', 'unsplash contest winner', 'utagawa kuniyoshi', 'van gogh', 'vaporwave', 'velvia', 'vermeer', 'vfxfriday', 'victo ngai', 'victor adame minguez', 'victor moscoso', 'victor mosquera', 'vincent di fate', 'vincent proce', 'vint lawrence', 'violet oakley', 'virginia lee burton', 'vladimir kush', 'vladimir tretchikoff', 'volkan baga', 'volumetric lighting', 'volumetric', 'voxelized', 'vray tracing', 'vray', 'wally wood', 'walt reed', 'walter beach humphrey', 'walter crane', 'walter humphrey', 'warhol', 'warren mahy', 'wassily kandinsky', 'watercolor', 'wavy', 'wayne england', 'wayne reynolds', 'weiwei', 'wendell minor', 'wes wilson', 'wesley burt', 'whitney darrow jr', 'whitney sherman', 'wifredo lam', 'will eisner', 'willard mullin', 'william blake', 'william glackens', 'william michael harnett', 'william steig', 'william stout', 'william turner', 'willian murai', 'willie ito', 'willy art', 'windows xp', 'winona nelson', 'winsor mccay', 'wojciech siudmak', 'woodcut', 'wylie beckert', 'xbox 360 graphic', 'y2k aesthetic', 'yasar vurdem', 'yasutomo oka', 'yayoi kusama', 'yayou kusama', 'yeong hao han', 'yoshitaka amano', 'yoshitoshi abe', 'yue minjun', 'yukimasa ida', 'yuko shimizu', 'yuko tatsushima', 'yumihiko amano', 'yves tanguy', 'zacharias martin aagaard', 'zack stella', 'zbrush central contest winner', 'zbrush', 'zdzislaw beksinski', 'zhang zeduan', 'zhou wenjing', 'zo mozert', 'zoltan boros']
import sys
sys.path.append("pixray")
import pixray
for i in keywords[45:50]:
  ## Simple setup from menu choices
  pixray.reset_settings()
  pixray.add_settings(prompts="powerful sigil magic symbol to energise you spirit by greg rutkowski and thomas kinkade, Trending on artstation.") ##"Father by "+str(i)+" )
  pixray.add_settings(drawer="vqgan", vqgan_model="wikiart_16384", clip_models="RN50, ViT-B/16, ViT-B/32")
  pixray.add_settings(quality="draft")
  pixray.add_settings(iterations=200, size = [200,200])
  pixray.add_settings(display_clear=True)
  pixray.add_settings(outdir="/content/drive/MyDrive/pixray_generated_art")
  pixray.add_settings(output=str(i)+".png")
  settings = pixray.apply_settings()
  pixray.do_init(settings)
  pixray.do_run(settings)

# hey, let's try that new RN50x16 CLIP model huh?
#pixray.add_settings(clip_models="RN50x16")

# examples below start with '#" are comments and do not get run
# pixray.add_settings(init_image="file.png")
# pixray.add_settings(init_image_alpha=200)
# pixray.add_settings(overlay_image="file.png")
# pixray.add_settings(image_prompts="file.png")
# pixray.add_settings(overlay_every=20)
# pixray.add_settings(seed=42)
# pixray.add_settings(display_every=50)
# pixray.add_settings(iterations=200)
# pixray.add_settings(size=[256, 256])
# pixray.add_settings(vector_prompts="textoff:4")

# if you enable drawers below make sure to disable the "vqgan_model" line above
# pixray.add_settings(drawer="fft")
# pixray.add_settings(drawer="pixel")
# pixray.add_settings(drawer="vdiff")

# change from CLIP guided to SLIP guided by enabling this line
# pixray.add_settings(perceptors="slip")

In [ ]:
#@title Now Run

#@markdown Now we'll apply all settings and run.
#@markdown 
#@markdown Click the play button to the left to get this running.
#@markdown 
#@markdown When this is done, you can go above and change the
#@markdown model in the settings and see if you get different results.
#@markdown Note that the first time you run any model it takes a bit
#@markdown longer because the model has to download when needed.

settings = pixray.apply_settings()
pixray.do_init(settings)
pixray.do_run(settings)
